In [146]:
import pandas as pd
import numpy as np

import _pickle as pickle
from collections import Counter
from pprint import pprint
from datetime import datetime
import shutil
import time
import os

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
ROOT_DIR = "../"

DATA_DIR = ROOT_DIR
RESULTS_DIR = ROOT_DIR + "results/"

# Train - 1460 (1168 + 292) videos 
# Test - 252 videos 

TRAINING_DATA = DATA_DIR + "video_training_data.p" # each read gives labelled video [(segment, label), segment_2, ...]
VALIDATION_DATA = DATA_DIR + "video_validation_data.p" 
TEST_DATA = DATA_DIR + "video_testing_data.p" # each read gives video [(segment), segment_2, ...]

## Load data from file

In [ ]:
# def transform_input(segments):
#     """
#     Transform input with zero padding so that all input segments 
#     in the batch have same size.
#     """
#     input_dim = 400 # num features in an i3D frame
#     segment_lengths = [len(segments) for segments in segments]
    
#     # Create an empty matrix with zero padding
#     longest_segment = max(segment_lengths)
#     batch_size = len(segments)
#     padded_segments = np.zeros((batch_size, longest_segment, input_dim))
    
#     # Copy over the actual frame sequences
#     for i, length in enumerate(segment_lengths):
#         sequence = segments[i]
#         padded_segments[i, 0:length] = sequence[:length]
    
#     # Change back to tensor of type double
#     transformed_segments = []
#     for padded_segment in padded_segments:
#         transformed_segments.append(torch.Tensor(padded_segment).double())
    
#     return transformed_segments

In [ ]:
# def get_next_data_batch(f, batch_size):    
#     raw_videos = []
#     is_end_reached = False
    
#     num_videos = 0
#     while num_videos < batch_size:
#         try:
#             video = pickle.load(f) # [(segment, label), ...]
            
#             raw_videos.append(video)
#             num_videos += 1
#         except (EOFError):
#             is_end_reached = True
#             break
    
#     if len(raw_videos) == 0:
#         if not is_end_reached:
#             print('Returning empty data, but end of file NOT reached')
#         return [], [], is_end_reached
        
# #     How to pad video??
# #     padded_segments = transform_input(raw_segments)
# #     segments = torch.stack(padded_segments).double()

#     return raw_videos, is_end_reached

In [3]:
def get_next_video_data(f):    
    is_end_reached = False
    
    try:
        video = pickle.load(f) # [(segment, label), ...]
    except (EOFError):
        is_end_reached = True
        return [], is_end_reached
    
    return video, is_end_reached

## Model Architecture

In [344]:
class ClassPredictor(nn.Module):
    def __init__(self, input_size, num_classes):
        super(ClassPredictor, self).__init__()
        
        hidden_1 = 100
        hidden_2 = 40
        hidden_3 = 15
        
        self.temp = nn.Linear(input_size, num_classes)
        
        # https://pytorch.org/docs/stable/nn.html#torch.nn.Linear
        self.fc1 = nn.Linear(input_size, hidden_1)
        self.bn1 = nn.BatchNorm1d(num_features=hidden_1)
        self.l_relu1 = nn.LeakyReLU()
        self.dout1 = nn.Dropout(0.3)
        
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.bn2 = nn.BatchNorm1d(num_features=hidden_2)
        self.l_relu2 = nn.LeakyReLU()
        self.dout2 = nn.Dropout(0.2)
        
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.bn3 = nn.BatchNorm1d(num_features=hidden_3)
        self.l_relu3 = nn.LeakyReLU()
        
        self.out = nn.Linear(hidden_3, num_classes)
        
        with torch.no_grad():
            nn.init.xavier_uniform_(self.fc1.weight, gain=nn.init.calculate_gain('leaky_relu'))
            nn.init.xavier_uniform_(self.fc2.weight, gain=nn.init.calculate_gain('leaky_relu'))
            nn.init.xavier_uniform_(self.fc3.weight, gain=nn.init.calculate_gain('relu'))
            nn.init.xavier_uniform_(self.out.weight)
            
    def forward(self, x):
        # x: (input_size)
        return self.temp(x)
        
        a1 = self.fc1(x)
#         b1 = self.bn1(a1)
        h1 = self.l_relu1(a1)
        
        dout1 = self.dout1(h1)
        a2 = self.fc2(dout1)
#         b2 = self.bn2(a2)
        h2 = self.l_relu2(a2)
        
        dout2 = self.dout2(h2)
        a3 = self.fc3(dout2)
#         b3 = self.bn3(a3)
        h3 = self.l_relu3(a3)
        
        # y: (num_classes)
        y = self.out(h3)

        return y

In [354]:
# Bidirectional recurrent neural network (many-to-one)
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM
        self.bilstm = nn.LSTM(input_size, hidden_size, num_layers, bidirectional=True, 
                            batch_first=True, bias=True, dropout=0)
        
        # DNN for class prediction
        self.fc = ClassPredictor(hidden_size * 2, num_classes)
 

    def init_hidden_state(self, batch_size):
        #h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size)
        #c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size)

        # https://pytorch.org/docs/stable/nn.init.html

        h0 = torch.empty(self.num_layers * 2, batch_size, self.hidden_size).double()
        nn.init.orthogonal_(h0) # orthogonal_, xavier_normal_, xavier_uniform_
        
        c0 = torch.empty(self.num_layers * 2, batch_size, self.hidden_size).double()
        nn.init.orthogonal_(c0) # orthogonal_, xavier_normal_, xavier_uniform_

        h0.requires_grad_()#.to(device)
        c0.requires_grad_()#.to(device)
        
        return h0, c0
 

    def forward(self, x, segment_indices):
        # x: (batch_size, seq_len, feature_len)
        # segment_indices: (num_segments, 2)
        
        batch_size = x.size(0)

        # Set initial states
        # h0, c0: (num_layers * num_directions, batch_size, hidden_size)
        h0, c0 = self.init_hidden_state(batch_size)
        
        # Forward propagate LSTM
        # out: tensor of shape (batch_size, seq_length, hidden_size * 2)
        out, _ = self.bilstm(x, (h0, c0))  
        
        # Use hidden states of each segment to predict their labels
        segment_outputs = []
        for (start, end) in segment_indices:
            hidden_states = out[:, start:end, :]
            # Compute the hidden state by doing temporal pooling over all time steps
            # pool_out: (hidden_size * 2)

            # TODO: concat pooling.
            # https://medium.com/@sonicboom8/sentiment-analysis-with-variable-length-sequences-in-pytorch-6241635ae130
            max_pool_out = F.adaptive_max_pool1d(hidden_states.permute(0,2,1), 1).squeeze()

    #         avg_pool_out = F.adaptive_avg_pool1d(out.permute(0,2,1), 1).squeeze()
    #         final_hidden_out = out[:,-1, :]
            
            # output: (num_classes)
            output = self.fc(max_pool_out)
            
            segment_outputs.append(output)

        
        # segment_outputs: (num_segments, num_classes)
        segment_outputs = torch.stack(segment_outputs)
        
        return segment_outputs

## Train model on entire videos (sequence of segments which are sequence of frames)

In [347]:
def transform_to_inputs(video):
    segments = [] # segments (list of frames) in the video
    labels = [] # labels of each segment
    segment_indices = []
    offset = 0
    for segment_num in range(len(video)):
        segments.append(video[segment_num][0])
        labels.append(video[segment_num][1] - 1) # NOTE: subtract 1 since SIL (class 0) is not used
        segment_indices.append((offset, offset + video[segment_num][0].shape[0]))

        offset += video[segment_num][0].shape[0]
        
    # Load frames as tensors with gradient accumulation abilities
    input_frames = torch.cat(segments, 0).unsqueeze(0).requires_grad_() # unsqueeze to add batch dim
    labels = torch.Tensor(labels).long()
    
    return input_frames, labels, segment_indices

In [348]:
def save_checkpoint(state, filename):
    torch.save(state, filename)

def train_model(model, num_epochs=10, train_data_file=TRAINING_DATA, validation_data_file=VALIDATION_DATA):
    training_losses = []
    validation_losses = []

    
    for epoch in range(num_epochs):
        start = time.time()
        print('======================================================================')
        print('Epoch: {}'.format(epoch))

        try:
            train_f.close()    
        except(NameError):
            ;   
        
        train_f = open(train_data_file, 'rb')
        should_epoch_end = False

        video_num = 0
        epoch_training_losses = []

        model.train()
        while not should_epoch_end:
            video, should_epoch_end = get_next_video_data(train_f)
            if should_epoch_end:
                break
            
            inputs, labels, segment_indices = transform_to_inputs(video)

            outputs = model(inputs, segment_indices)
            loss = loss_criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_training_losses.append(loss.item())

            video_num += 1
            if video_num % 200 == 0:
                print('    Iteration: {}. Loss: {}.'.format(iter, loss.item())) 

    
        train_f.close()            
        
        # Evaluate on validation set
        try:
            val_f.close()    
        except(NameError):
            ;  
            
        val_f = open(validation_data_file, 'rb')
        model.eval()

        correct = 0
        total = 0
        is_file_end = False

        epoch_validation_losses = []
        while not is_file_end:
            video, is_file_end = get_next_video_data(train_f)
            if is_file_end:
                break
            
            inputs, labels, segment_indices = transform_to_inputs(video)
        
            outputs = model(inputs, segment_indices)
            loss = loss_criterion(outputs, labels)
            epoch_validation_losses.append(loss.item())

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        val_f.close()
        

        accuracy = 100 * correct / total
        
        # Print loss
        training_losses.append(np.mean(epoch_training_losses))
        validation_losses.append(np.mean(epoch_validation_losses))

        scheduler.step(validation_losses[-1])

        print(f' Training Loss: {training_losses[-1]} Validation Loss: {validation_losses[-1]} Validation Accuracy: {accuracy}%')
        end = time.time()
        print(f'Epoch took {end - start} seconds')
        
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer' : optimizer.state_dict(),
        }, RESULTS_DIR + model_time + '/model_checkpoint.pth')
        
    return training_losses, validation_losses

In [ ]:
model_time = str(datetime.now().strftime("%Y-%m-%d_%I-%M-%S"))
os.mkdir(model_time)

## Segment (inner) Model Architecture
input_dim = 400  # dimension of an i3D video frame
hidden_dim = 100 # dimension of hidden state
layer_dim = 1    # number of stacked layers
output_dim = 47  # number of sub-action labels


model = BiLSTM(input_dim, hidden_dim, layer_dim, output_dim)
model = model.double() # transform the model parameters to double precision


## Loss function
loss_criterion = nn.CrossEntropyLoss()


## Optimizer
learning_rate = 0.01
weight_decay = 0.002
momentum = 0.9

optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay) 


## Learning Rate Scheduler
patience = 3
decrease_factor = 0.5
min_learning_rate = 0.00005
scheduler = ReduceLROnPlateau(optimizer, mode='min', 
                              patience=patience, min_lr=min_learning_rate, factor=decrease_factor,
                              verbose=True)


num_epochs = 3

training_losses, validation_losses = train_model(model,num_epochs=num_epochs) # TODO: batch_sizes

path = RESULTS_DIR + model_time + "/rnn_model.pth"
torch.save(model.state_dict(), path)
print("\nSaved trained model to", path)

Epoch: 0


In [ ]:
# Plot training and validation losses
# training_losses = training_losses.mean()
# validation_losses = validation_losses.mean()

plt.figure(figsize=(12,6))
plt.title("Average batch Training and validation loss in each epoch")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(training_losses)
plt.plot(validation_losses)
plt.legend(["Training loss","Validation loss"])

In [ ]:
# # Load saved checkpoint
# checkpoint_path = RESULTS_DIR + "2020-03-21-_04-30-14" + "/model_checkpoint.pth"
# checkpoint = torch.load(checkpoint_path)

# epoch = checkpoint['epoch']
# model.load_state_dict(checkpoint['state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer'])

# print("=> loaded checkpoint '{}' (epoch {})".format(checkpoint_path, epoch))

# # # Load saved model
# # path = RESULTS_DIR + "2020-03-21-_04-30-14" + "/rnn_model.pth"
# # model.load_state_dict(torch.load(path))

# Evaluate model on validation set

In [ ]:
model.eval()

try:
    val_f.close()    
except(NameError):
    ;   
val_f = open(VALIDATION_DATA, 'rb')

y = {'Actual': [], 'Predicted': []}

is_end_reached = False
while not is_end_reached:
    segments, labels, is_end_reached = get_next_data_batch(val_f, 1)
    
    if len(segments) == 0:
        break
    
    outputs = model(segments)
    _, predicted = torch.max(outputs.data, 1)
    
    y['Actual'].extend(labels.tolist())
    y['Predicted'].extend(predicted.tolist())
    
y_df = pd.DataFrame(y)
path = RESULTS_DIR + model_time + "/prediction_results.csv"        
y_df.to_csv(path, encoding='utf-8', index=False)

print('Done predicting on validation set')

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score

print('\nMacro f1 Score= %.4f' % f1_score(y['Actual'], y['Predicted'], average="macro"))
print('Macro Precision= %.4f' % precision_score(y['Actual'], y['Predicted'], zero_division=0, average="macro"))
print('Macro Recall= %.4f' % recall_score(y['Actual'], y['Predicted'], average="macro")) 

print('\nMicro f1 Score= %.4f' % f1_score(y['Actual'], y['Predicted'], average="micro"))
print('Micro Precision= %.4f' % precision_score(y['Actual'], y['Predicted'], zero_division=0, average="micro"))
print('Micro Recall= %.4f' % recall_score(y['Actual'], y['Predicted'], average="micro")) 

print('\nAccuracy: %.4f' % accuracy_score(y['Actual'], y['Predicted']))

# Computes the average AUC of all possible pairwise combinations of classes. 
# Insensitive to class imbalance when average == 'macro'.
# print(roc_auc_score(y['Actual'], y['Predicted'], multi_class='ovo', average='macro')) 

# Generate test data predictions for submission

In [ ]:
NUM_TEST_SEGMENTS = 1284

def get_next_test_batch(f, batch_size=1):    
    raw_segments = []
    
    is_end_reached = False
    
    num_segments = 0
    while num_segments < batch_size:
        try:
            segment = pickle.load(f)
            
            raw_segments.append(segment)
            num_segments += 1
        except (EOFError):
            is_end_reached = True
            break

    padded_segments = transform_input(raw_segments)
    segments = torch.stack(padded_segments).double()
    
    return segments, is_end_reached


model.eval()

try:
    test_f.close()    
except(NameError):
    ;   
test_f = open(TEST_DATA, 'rb')

y_pred = {'Id': np.arange(NUM_TEST_SEGMENTS), 'Category': []}

# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/bidirectional_recurrent_neural_network/main.py

is_end_reached = False
while not is_end_reached:
    segments, is_end_reached = get_next_test_batch(test_f, 50)

    outputs = model(segments)
    _, predicted = torch.max(outputs.data, 1)
    
    y_pred['Category'].extend(predicted.tolist())

y_pred_df = pd.DataFrame(y_pred)
path = RESULTS_DIR + model_time + "/predictions_submission.csv"
y_pred_df.to_csv(path, encoding='utf-8', index=False)

print('Done predicting on test data')